<a href="https://colab.research.google.com/github/donggale72/10000.html/blob/main/5_2_%EC%BC%80%EB%9D%BC%EC%8A%A4_%EC%B0%BD%EC%8B%9C%EC%9E%90%EC%97%90%EA%B2%8C_%EB%B0%B0%EC%9A%B0%EB%8A%94_%EB%94%A5%EB%9F%AC%EB%8B%9DImageDataGenerator%EC%97%B0%EC%8A%B5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/yebiny/SkillTreePython-DeepLearning


Cloning into 'SkillTreePython-DeepLearning'...
remote: Enumerating objects: 1007, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 1007 (delta 65), reused 55 (delta 30), pack-reused 906
Receiving objects: 100% (1007/1007), 56.03 MiB | 25.94 MiB/s, done.
Resolving deltas: 100% (544/544), done.


In [ ]:
import sys
sys.path.append('/content/SkillTreePython-DeepLearning/02.케라스_창시자에게_배우는_딥러닝/scripts')
from import_lib import *

In [ ]:
! git clone https://github.com/gilbutITbook/006975

Cloning into '006975'...
remote: Enumerating objects: 102534, done.
remote: Total 102534 (delta 0), reused 0 (delta 0), pack-reused 102534
Receiving objects: 100% (102534/102534), 202.75 MiB | 23.82 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (104042/104042), done.


In [ ]:
import glob
org_img_paths=sorted(glob.glob('/content/006975/datasets/cats_and_dogs/train/*jpg'))
len(org_img_paths)

4000

In [ ]:
os.mkdir('new_ds')

In [ ]:
for dir_type in ['train', 'valid', 'test']:
  dir_path=f'new_ds/{dir_type}'
  os.mkdir(dir_path)
  print(f'*{dir_path} is made')
  for dir_label in ['cat', 'dog']:
    dir_path=f'new_ds/{dir_type}/{dir_label}'
    os.mkdir(dir_path)
    print(f'*ㄴ{dir_path}is made')


*new_ds/train is made
*ㄴnew_ds/train/catis made
*ㄴnew_ds/train/dogis made
*new_ds/valid is made
*ㄴnew_ds/valid/catis made
*ㄴnew_ds/valid/dogis made
*new_ds/test is made
*ㄴnew_ds/test/catis made
*ㄴnew_ds/test/dogis made


In [ ]:
import shutil
for img_path in org_img_paths:
  info=img_path.split('/')[-1]
  label, idx, _=info.split('.')

  if 0<=int(idx)<1000: target_path=f'/content/new_ds/train/{label}/{info}'
  elif 1000<=int(idx)<1500: target_path=f'/content/new_ds/valid/{label}/{info}'
  else: target_path=f'/content/new_ds/test/{label}/{info}'
  shutil.copyfile(img_path, target_path)

In [ ]:
for dir_type in ['train', 'valid', 'test']:
  dir_path=f'new_ds/{dir_type}'
  for dir_label in ['cat', 'dog']:
    dir_path=f'new_ds/{dir_type}/{dir_label}'
    n_imgs=len(os.listdir(dir_path))
    print(f'*{dir_path} 이미지 전체 개수: {n_imgs}')

*new_ds/train/cat 이미지 전체 개수: 1000
*new_ds/train/dog 이미지 전체 개수: 1000
*new_ds/valid/cat 이미지 전체 개수: 500
*new_ds/valid/dog 이미지 전체 개수: 500
*new_ds/test/cat 이미지 전체 개수: 500
*new_ds/test/dog 이미지 전체 개수: 500


In [ ]:
def create_model():
  x = layers.Input(shape=(150,150, 3)) # resolution 축소 및 크기를 동일하게 맞춰주는 작업 필요
  y = layers.Conv2D(32, 3, activation='relu')(x)
  y = layers.MaxPool2D()(y)
  y = layers.Conv2D(64, 3, activation='relu')(y)
  y = layers.MaxPool2D()(y)
  y = layers.Conv2D(128, 3, activation='relu')(y)
  y = layers.MaxPool2D()(y)
  y = layers.Conv2D(128, 3, activation='relu')(y)
  y = layers.MaxPool2D()(y)
  y = layers.Flatten()(y)
  y = layers.Dense(512, activation='relu')(y)
  y = layers.Dense(1, activation = 'sigmoid')(y)
  return models.Model(x, y)

In [ ]:
model=create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856 

In [ ]:
from tensorflow.keras import losses, optimizers
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=optimizers.RMSprop(learning_rate=0.0001),
              metrics=['acc'])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen=ImageDataGenerator(rescale=1.0/255.0).flow_from_directory('/content/new_ds/train',
                                                                    target_size=(150,150),
                                                                    batch_size=20,
                                                                    class_mode='binary')
valid_gen=ImageDataGenerator(rescale=1.0/255.0).flow_from_directory('/content/new_ds/valid',
                                                                    target_size=(150,150),
                                                                    batch_size=20,
                                                                    class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
for (x_batch, y_batch) in train_gen:
  print(f'배치 X 크기: {x_batch.shape}')
  print(f'배치 Y 크기: {y_batch.shape}')
  break

배치 X 크기: (20, 150, 150, 3)
배치 Y 크기: (20,)


In [ ]:
history=model.fit(train_gen, 
                  steps_per_epoch=100,
                  epochs=30,
                  validation_data=valid_gen,
                  validation_steps=50)

Epoch 1/30
100/100 [==============================] - 29s 122ms/step - loss: 0.6923 - acc: 0.5315 - val_loss: 0.7049 - val_acc: 0.5000
Epoch 2/30
100/100 [==============================] - 10s 96ms/step - loss: 0.6616 - acc: 0.5965 - val_loss: 0.6415 - val_acc: 0.6340
Epoch 3/30
100/100 [==============================] - 10s 96ms/step - loss: 0.6067 - acc: 0.6755 - val_loss: 0.6090 - val_acc: 0.6670
Epoch 4/30
100/100 [==============================] - 10s 96ms/step - loss: 0.5757 - acc: 0.7040 - val_loss: 0.6292 - val_acc: 0.6470
Epoch 5/30
100/100 [==============================] - 9s 95ms/step - loss: 0.5455 - acc: 0.7215 - val_loss: 0.5978 - val_acc: 0.6810
Epoch 6/30
100/100 [==============================] - 10s 96ms/step - loss: 0.5184 - acc: 0.7470 - val_loss: 0.5747 - val_acc: 0.7060
Epoch 7/30
100/100 [==============================] - 10s 96ms/step - loss: 0.4863 - acc: 0.7650 - val_loss: 0.5475 - val_acc: 0.7260
Epoch 8/30
100/100 [==============================] - 10s 96ms

In [ ]:
model.save('model.h5')


In [ ]:
from plot_result import *
plot_lcurve([history], [''], ['dodgerblue'], x_itv=5)


In [ ]:
aug_gen= ImageDataGenerator(rescale=1.0/255.0,
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest',
                            brightness_range=[0.7,1.3]
                            )

In [ ]:
from tensorflow.keras.preprocessing import image

img=image.load_img(org_img_paths[4], target_size=(150,150))
x=image.img_to_array(img)
x=np.expand_dims(x, 0)
x.shape

In [ ]:
for i, x_aug in enumerate(aug_gen.flow(x, batch_size=1)):
  plt.subplot(2,3,i+1)
  plt.imshow(x_aug[0])
  plt.xticks([]); plt.yticks([])
  if i==5: break

In [ ]:
def create_model2():
  x = layers.Input(shape=(150,150, 3)) # resolution 축소 및 크기를 동일하게 맞춰주는 작업 필요
  y = layers.Conv2D(32, 3, activation='relu')(x)
  y = layers.MaxPool2D()(y)
  y = layers.Conv2D(64, 3, activation='relu')(y)
  y = layers.MaxPool2D()(y)
  y = layers.Conv2D(128, 3, activation='relu')(y)
  y = layers.MaxPool2D()(y)
  y = layers.Conv2D(128, 3, activation='relu')(y)
  y = layers.MaxPool2D()(y)
  y = layers.Flatten()(y)
  y = layers.Dense(512, activation='relu')(y)
  y = layers.Dense(1, activation = 'sigmoid')(y)
  return models.Model(x, y, name='model2')

model2=create_model2()
model2.summary()

from tensorflow.keras import losses, optimizers
model2.compile(loss=losses.BinaryCrossentropy(),
               optimizer=optimizers.RMSprop(learning_rate=0.0001),
               metrics=['acc'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen=aug_gen.flow_from_directory('/content/new_ds/train',
                                      target_size=(150,150),
                                      batch_size=20,
                                      class_mode='binary')
valid_gen=ImageDataGenerator(rescale=1.0/255.0).flow_from_directory('/content/new_ds/valid',
                                                                    target_size=(150,150),
                                                                    batch_size=20,
                                                                    class_mode='binary')

history2=model2.fit(train_gen, 
                    steps_per_epoch=100,
                    epochs=50,
                    validation_data= valid_gen,
                    validation_steps=50)

In [ ]:
model2.save('cats_and_dogs_small.h5')

In [ ]:
plot_lcurve([history2],[''],['dodgerblue'])

https://www.tensorflow.org/tutorials/images/classification?hl=ko

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, utils
from tensorflow.keras.models import Sequential

In [ ]:
import pathlib
dataset_url='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
data_dir=tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir=pathlib.Path(data_dir)

In [ ]:
image_count=len(list(data_dir.glob('*/*.jpg')))
print(image_count)


In [ ]:
roses=list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
PIL.Image.open(str(roses[1]))

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
PIL.Image.open(str(tulips[1]))

In [ ]:
batch_size=32
img_height=180
img_width=180


In [ ]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir, 
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
class_names=train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax=plt.subplot(3,3, i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(class_names[labels[i]])
    plt.axis('off')

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE=tf.data.experimental.AUTOTUNE

train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds=val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer=layers.experimental.preprocessing.Rescaling(1./255.)

In [ ]:
normalized_ds=train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch=next(iter(normalized_ds))
first_image=image_batch[0]
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes=5

model=Sequential([
                  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
                  layers.Conv2D(16,3, padding='same',activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Conv2D(32,3, padding='same', activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Conv2D(64,3, padding='same', activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Flatten(),
                  layers.Dense(128, activation='relu'),
                  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range=range(epochs)

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:

data_augmentation=keras.Sequential(
    [
    layers.experimental.preprocessing.RandomFlip('horizontal',
                                                input_shape=(img_height,
                                                             img_width,
                                                             3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1) ]
)

In [ ]:
plt.figure(figsize=(10,10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images=data_augmentation(images)
    ax=plt.subplot(3,3,i+1)
    plt.imshow(augmented_images[0].numpy().astype('uint8'))
    plt.axis('off')

In [ ]:
model=Sequential([
                  data_augmentation,
                  layers.experimental.preprocessing.Rescaling(1./255),
                  layers.Conv2D(16,3, padding='same',activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Conv2D(32,3, padding='same', activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Conv2D(64,3, padding='same', activation='relu'),
                  layers.MaxPooling2D(),
                  layers.Dropout(0.2),
                  layers.Flatten(),
                  layers.Dense(128, activation='relu'),
                  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=15
history=model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range=range(epochs)

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.figure(figsize=(8,8))
plt.subplot(1,2,2)
plt.plot(epochs_range, acc, label='Training Loss')
plt.plot(epochs_range, val_acc, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()



In [ ]:
sunflower_url='https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg'
sunflower_path=tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img=keras.preprocessing.image.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array=keras.preprocessing.image.img_to_array(img)
img_array=tf.expand_dims(img_array, 0)

predictions=model.predict(img_array)
score=tf.nn.softmax(predictions[0])

print(
    'This image most likely belongs to {} with a{:.2f} percent confidence.'.format(class_names[np.argmax(score)], 100*np.max(score))
)